In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

In [2]:
LOCAL_LLM = 'llama3'
CLOUD_LLM = 'llama3-70b-8192'
VECTOR_DB_URL = 'http://localhost:6333'
EMBEDDING_MODEL = 'NeuML/pubmedbert-base-embeddings'

In [3]:
positive_query = """
Given the following subjective and objective assessment, provide a well informed and researched differential diagnosis

Subjective assessment:
The patient, Mr. Smith, is a 45-year-old male who presents to the clinic with complaints of lower back pain that has been bothering him for the past two weeks. He describes the pain as dull and achy, located in the lumbar region, with occasional radiation down his left leg. He notes that the pain worsens with prolonged sitting or standing and is relieved by lying down. He denies any recent trauma or injury but mentions that he has a history of occasional low back pain, especially after heavy lifting or prolonged periods of inactivity. He rates the pain as a 6 out of 10 on the pain scale.

Objective assessment:
On physical examination, Mr. Smith appears uncomfortable but is able to walk into the examination room without assistance. Vital signs are within normal limits. Inspection of the lumbar spine reveals no obvious deformities or asymmetry. Palpation elicits tenderness over the paraspinal muscles of the lumbar spine, particularly on the left side. Range of motion of the lumbar spine is mildly restricted, with pain on forward flexion and left lateral bending. Straight leg raise test is positive on the left side at 45 degrees, reproducing his symptoms of radiating pain down the left leg. Neurological examination reveals intact sensation and strength in the lower extremities, with no signs of motor weakness or sensory deficits.
"""

In [4]:
from langchain_community.vectorstores.qdrant import Qdrant
from qdrant_client import QdrantClient
from langchain_community.embeddings import sentence_transformer

def get_vector_embeddings(embedding_model: str):
    embeddings = sentence_transformer.SentenceTransformerEmbeddings(
        model_name=embedding_model
    )
    return embeddings

print("Loading vector embeddings and creating Qdrant client...")
embeddings = get_vector_embeddings(EMBEDDING_MODEL)
client = QdrantClient(VECTOR_DB_URL)
db = Qdrant(
    client=client,
    embeddings=embeddings,
    collection_name="test-collection",
)

Loading vector embeddings and creating Qdrant client...


/Users/dingruoqian/code/TLM3001-DP-PhysioLLM/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from langchain_core.output_parsers.json import JsonOutputParser
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field
from typing_extensions import TypedDict
from typing import List
from langchain.schema import Document
from pprint import pprint
from langgraph.graph import StateGraph, END

In [6]:
### Retrieval Grader

# Create pydantic object for the grader result
class GraderResult(BaseModel):
    score: str = Field(
        "", 
        description="A binary score 'yes' or 'no' to indicate whether the document is relevant to the question."
    )

parser = PydanticOutputParser(pydantic_object=GraderResult)

RETRIEVAL_GRADER_PROMPT = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|> 
You are a grader assessing relevance of a retrieved document to a user question. 
If the document contains keywords related to the user question,
grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals.

{format_instructions}<|eot_id|>
<|start_header_id|>user<|end_header_id|>

Here is the retrieved document:

{document}

Here is the user question: {question}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

llm = ChatGroq(model=CLOUD_LLM)
# llm = ChatOllama(model=LOCAL_LLM, format="json", temperature=0)

prompt = PromptTemplate(
    template=RETRIEVAL_GRADER_PROMPT, 
    input_variables=["document", "question"], 
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

retrieval_grader = prompt | llm | JsonOutputParser()
retrieval_grader = retrieval_grader.with_retry()

query = "When did sir Standford Raffles founded singapore?"
docs = db.similarity_search_with_score(query=query, k=5)

# Test the retrieval grader agent
for i , (doc, score) in enumerate(docs, start=1):
    result = retrieval_grader.invoke({
        "document": doc,
        "question": query
    })
    print(doc.page_content)
    print(f"{i} - {result}", sep="\n\n", end="\n\n\n" + "-" * 50 + "\n\n\n")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


of the now departed “giants” Professors John Pegington (University College London), Sandy Marks (University of Massachusetts, USA) and Hanno Boon (Pretoria, South Africa) working with myself (PHA). For previous dedications see the sixth edition dedication online (www.studentconsult.com).In the autumn of 2012 we heard the sad news of Bob McMinn’s passing at the age of 88. Following in his father’s footsteps Bob, graduated from Glasgow University in medi-cine in 1947. His main academic career was in London, ﬁrst as Professor at Kings College, London and then as the William Collins Professor at the Royal College of Surgeons of England. Along the way Bob not only gained an MD but a PhD as well in the ﬁeld of wound healing and tissue repair. However, it is for this revolutionary McMinn’s Colour Atlas of Human Anatomy, ﬁrst produced in 1977, that Bob’s name is known worldwide. Not only will this seventh edition bring sales to over 2 million in some 30 languages, including Latin, Korean, Chin

In [7]:
### Generation Agent

from langchain_core.output_parsers.string import StrOutputParser

PROMPT = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a expert in physiotherapy, you will be presented with a set of subjective and object assessment and your job is to come up with a well informed and researched differential diagnosis of the medical scenerio.
Here are examples of the subjective and objective assessments and the expected differential diagnosis:

Subjective Assessment:
The patient, Mr. Smith, is a 45-year-old male who presents to the clinic with complaints of lower back pain that has been bothering him for the past two weeks. He describes the pain as dull and achy, located in the lumbar region, with occasional radiation down his left leg. He notes that the pain worsens with prolonged sitting or standing and is relieved by lying down. He denies any recent trauma or injury but mentions that he has a history of occasional low back pain, especially after heavy lifting or prolonged periods of inactivity. He rates the pain as a 6 out of 10 on the pain scale.On physical examination, Mr. Smith appears uncomfortable but is able to walk into the examination room without assistance.

Objective Assessment:
Vital signs are within normal limits. Inspection of the lumbar spine reveals no obvious deformities or asymmetry. Palpation elicits tenderness over the paraspinal muscles of the lumbar spine, particularly on the left side. Range of motion of the lumbar spine is mildly restricted, with pain on forward flexion and left lateral bending. Straight leg raise test is positive on the left side at 45 degrees, reproducing his symptoms of radiating pain down the left leg. Neurological examination reveals intact sensation and strength in the lower extremities, with no signs of motor weakness or sensory deficits. On physical examination, Mr. Smith appears uncomfortable but is able to walk into the examination room without assistance. Vital signs are within normal limits. Inspection of the lumbar spine reveals no obvious deformities or asymmetry. Palpation elicits tenderness over the paraspinal muscles of the lumbar spine, particularly on the left side. Range of motion of the lumbar spine is mildly restricted, with pain on forward flexion and left lateral bending. Straight leg raise test is positive on the left side at 45 degrees, reproducing his symptoms of radiating pain down the left leg. Neurological examination reveals intact sensation and strength in the lower extremities, with no signs of motor weakness or sensory deficits.

Differential Diagnosis:
Based on the assessments provided, my differential diagnosis for Mr. Smith would include:
1. ...
2. ...
...

IMPORTANT:
- If the query is not related to physiotherapy or unrelated from the retrieved context, please answer with "I am sorry, I am not able to answer this question."
- Include citations and references to support your answer. <|eot_id|>
<|start_header_id|>user<|end_header_id|> 

Answer the question based only on the following context, this context should be used as source of ground truth to answer the question:

{context}

---

Answer the question based on the above context: 
{question}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

llm = ChatGroq(model=CLOUD_LLM)

prompt = PromptTemplate(template=PROMPT, input_variables=["context", "question"])
rag_chain = prompt | llm | StrOutputParser()

docs = db.similarity_search_with_score(query=positive_query, k=5)

def format_retrieved_docs(docs):
    return "\n\n---\n\n".join([f"metadata: {doc.metadata}\nscore: {score}\ncontent: {doc.page_content}" for doc, score in docs])

# Test the generation agent
print(prompt.format(context=format_retrieved_docs(docs), question=positive_query))
print("Waiting for LLM response...")
stream = rag_chain.stream({
  "context": format_retrieved_docs(docs),
  "question": positive_query
})
generation = ""
for response in stream:
    generation += response
    print(response, flush=True, end="")




<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a expert in physiotherapy, you will be presented with a set of subjective and object assessment and your job is to come up with a well informed and researched differential diagnosis of the medical scenerio.
Here are examples of the subjective and objective assessments and the expected differential diagnosis:

Subjective Assessment:
The patient, Mr. Smith, is a 45-year-old male who presents to the clinic with complaints of lower back pain that has been bothering him for the past two weeks. He describes the pain as dull and achy, located in the lumbar region, with occasional radiation down his left leg. He notes that the pain worsens with prolonged sitting or standing and is relieved by lying down. He denies any recent trauma or injury but mentions that he has a history of occasional low back pain, especially after heavy lifting or prolonged periods of inactivity. He rates the pain as a 6 out of 10 on the pain scale.On 

In [8]:
class HallucinationGraderResult(BaseModel):
    score: str = Field(
        "", 
        description="A binary score 'yes' or 'no' to indicate whether the answer is grounded in / supported by the facts provided."
    )

parser = PydanticOutputParser(pydantic_object=HallucinationGraderResult)

PROMPT = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether
an answer is grounded in / supported by as set of facts. Give a binary score of 'yes' or 'no' to indicate
whether the answer is grounded in / supported by the facts provided.
{format_instructions}<|eot_id|>
<|start_header_id|>user<|end_header_id|>

Here are the facts:

----------------------------------------
{facts}
----------------------------------------

Here is the answer: {answer}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

llm = ChatGroq(model=CLOUD_LLM)

prompt = PromptTemplate(
    template=PROMPT, 
    input_variables=["facts", "answer"], 
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

hallucination_grader = prompt | llm | JsonOutputParser()
hallucination_grader = hallucination_grader.with_retry()

hallucination_result = hallucination_grader.invoke({
    "facts": format_retrieved_docs(docs),
    "answer": generation
})
print(hallucination_result)

{'score': 'no'}


In [9]:
### Answer Grader Agent

class AnswerGraderResult(BaseModel):
    score: str = Field("", description="A binary score 'yes' or 'no' to indicate whether the answer is useful to resolve the question")

parser = PydanticOutputParser(pydantic_object=AnswerGraderResult)

PROMPT = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether
an answer is useful to resolve a question. Give a binary score of 'yes' or 'no' to indicate whether the answer is useful to resolve the question.
{format_instructions}<|eot_id|>
<|start_header_id|>user<|end_header_id|> 

Here is the answer:
----------------------------------------
{answer}
----------------------------------------

Here is the question: 
{question}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

llm = ChatGroq(model=CLOUD_LLM)

prompt = PromptTemplate(
    template=PROMPT, 
    input_variables=["answer", "question"], 
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
answer_grader = prompt | llm | JsonOutputParser()
answer_grader = answer_grader.with_retry()

answer_result = answer_grader.invoke({
    "answer": generation,
    "question": query
})
print(answer_result)

{'score': 'no'}


In [10]:
### Router Agent

class RouterResult(BaseModel):
    datasource: str = Field(
        default="",
        description="A binary answer 'vectorstore' or 'websearch' to indicate whether the question should be routed to the vectorstore or web search."
    )
parser = PydanticOutputParser(pydantic_object=RouterResult)


prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an expert at routing a 
    user question to a vectorstore or web search. 
    Use the vectorstore for questions on Muscle skeletal conditions, phyisotherapy domain and clinic evaluations.
    You do not need to be stringent with the keywords 
    in the question related to these topics. Otherwise, use web-search.
    {format_instructions}<|eot_id|>
    <|start_header_id|>user<|end_header_id|> 
    
    Question to route: {question} <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

llm = ChatGroq(model=CLOUD_LLM)

negative_question = "llm agent memory"
positive_question = positive_query

question_router = prompt | llm | JsonOutputParser()
print(question_router.invoke({"question": positive_question})) 
# Should return 'vectorstore' if the question is related to physiotherapy or 'websearch' if not

{'datasource': 'vectorstore'}


In [11]:
### Web Search Tool
from langchain_community.tools.tavily_search import TavilySearchResults
web_search_tool = TavilySearchResults(max_results=3)

In [12]:
### Langgraph State

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        web_search: whether to add search
        documents: list of documents
    """

    question: str
    generation: str
    web_search: str
    documents: List[str]

In [13]:
### Conditional entry point
def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """
    
    print("---ROUTE QUESTION---")
    question = state["question"]
    print(question)
    
    source = question_router.invoke({"question": question})
    
    print(source["datasource"])
    if source["datasource"] == "web_search":
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "websearch"
    elif source["datasource"] == "vectorstore":
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"
    return "websearch"

In [14]:
### Node

def retrieve(state):
    """
    Retrieve documents from vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = db.similarity_search_with_score(query=question, k=5)
    documents = [doc.page_content for doc, score in documents]
    return {"documents": documents, "question": question}

In [15]:
### Node

def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question
    If any document is not relevant, we will set a flag to run web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents and updated web_search state
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    web_search = "No"
    
    for doc in documents:
        score = retrieval_grader.invoke(
            {"document": doc ,"question": question, }
        )
        grade = score["score"]
        
        if grade.lower() == "yes": # Document relevant
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(doc)
        else: # Document not relevant
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            # We do not include the document in filtered_docs
            # We set a flag to indicate that we want to run web search
            web_search = "Yes"
            
    return {"documents": filtered_docs, "question": question, "web_search": web_search}


### Conditional Edge

def decide_to_generate(state):
    """
    Determines whether to generate an answer, or add web search

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    web_search = state["web_search"]

    if web_search == "Yes":
        print("---DECISION: NOT ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---")
        return "websearch"
    else:
        print("---DECISION: GENERATE---")
        return "generate"

In [16]:
def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}


In [17]:
def web_search(state):
    """
    Web search based based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to documents
    """

    print("---WEB SEARCH---")
    question = state["question"]
    documents = state["documents"]

    # Web search
    docs = web_search_tool.invoke({"query": question[:200]})
    web_results = [doc['content'] for doc in docs]
    
    if documents is not None:
        documents.extend(web_results)
    else:
        documents = web_results
        
    return {"documents": documents, "question": question}


In [18]:
### Conditional edge

def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    # Run agent to check hallucinations
    score = hallucination_grader.invoke({"facts": documents,"answer": generation})
    score = "yes"

    if score == "yes": # If no hallucinations, check if generation answers question
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question, "answer": generation})
        grade = score["score"]
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
        
    else: # Hallucinations in generation, re-try
        pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

In [19]:
### Create Graph and add nodes

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("websearch", web_search)  # web search
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generate

In [20]:
### Build graph

workflow.set_conditional_entry_point(
    route_question,
    {
        "websearch": "websearch",
        "vectorstore": "retrieve",
    },
)

workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "websearch": "websearch",
        "generate": "generate",
    },
)
workflow.add_edge("websearch", "generate")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "websearch",
    },
)


In [21]:
# Compile and run the graph
app = workflow.compile()

inputs = {"question": positive_query}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
pprint(value["generation"])

---ROUTE QUESTION---

Given the following subjective and objective assessment, provide a well informed and researched differential diagnosis

Subjective assessment:
The patient, Mr. Smith, is a 45-year-old male who presents to the clinic with complaints of lower back pain that has been bothering him for the past two weeks. He describes the pain as dull and achy, located in the lumbar region, with occasional radiation down his left leg. He notes that the pain worsens with prolonged sitting or standing and is relieved by lying down. He denies any recent trauma or injury but mentions that he has a history of occasional low back pain, especially after heavy lifting or prolonged periods of inactivity. He rates the pain as a 6 out of 10 on the pain scale.

Objective assessment:
On physical examination, Mr. Smith appears uncomfortable but is able to walk into the examination room without assistance. Vital signs are within normal limits. Inspection of the lumbar spine reveals no obvious deform

BadRequestError: Error code: 400 - {'error': {'message': 'Please reduce the length of the messages or completion.', 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}